In [1]:
import seaborn
import pandas as pd
import dataframe_image
import datetime
import matplotlib as plt

In [2]:
def openFile(path):
    data = pd.read_csv(path)
    return data

In [3]:
def cleanUp (data, dropColumns):
    data = data.drop(columns=dropColumns)
    #data['Loaded'] = pd.to_datetime(data['Loaded'],format='%mm-%dd-%YYYY')
    #data['LastUpdated'] = pd.to_datetime(data['LastUpdated'],format='%mm-%dd-%YYYY')
    data = data.groupby('Id')
    return data

In [4]:
def getTotal(data):
    return len(data)

In [5]:
def getBusinessExceptions(data):
    exceptions = data.apply(lambda group: group['Tags'].str.contains('Business Exception').sum(),include_groups=False)
    return exceptions.sum()

In [6]:
def getCompletions(data, keyword):
    return data['Status'].apply(lambda status: status == keyword).sum()

In [7]:
def printData(data):
    totalItems = getTotal(data)
    businessExceptions = getBusinessExceptions(data)
    completions = getCompletions(data,"Complete")
    exceptions = totalItems - (completions + businessExceptions)
    print(f"Total items = {totalItems}")
    print(f"Completions = {completions}")
    print(f"Business exceptions = {businessExceptions}")
    print(f"Exceptions = {exceptions}")
    

In [54]:
def setStatus(data):
    if pd.notnull(data["Tags"]) and "Business Exception" in data["Tags"]:
        data["Status"] = "Business Exception"
    elif data["isLocked"] == 1:
        data["Status"] = "Locked"
    elif data["Status"] == "Complete":
        pass
    elif data["Status"] == "Pending":
        pass
    else:
        data["Status"] = "Exception"
        
        

    return data

In [70]:
def getStatusCount(data,status):
    count, _ = data[data["Status"] == status].shape
    return str(count)

In [71]:
directory = "resources/reports/08-26/"
filename = "icd.csv"
path = directory + filename
data = openFile(path)
data['isLocked'] = data['Locked'].apply(lambda x: 1 if pd.notnull(x) else 0)
drop = ['Resource', 'Loaded','LastUpdated', 'Exception', 'Deferred','Completed','KeyValue','Priority','Attempt','Worktime','Locked','ExceptionReason']
data = data.drop(columns=drop)
data["process"] = "ICD"
data['Status'] = data['Status'].apply(lambda x: x if pd.notnull(x) else "Pending")
data = data.apply(setStatus, axis=1)
print("Complete: " + getStatusCount(data,"Complete"))
print("Business Exception: " + getStatusCount(data,"Business Exception"))
print("Locked: " + getStatusCount(data,"Locked"))
print("Pending " + getStatusCount(data,"Pending"))
print("Exception " + getStatusCount(data,"Exception"))


Complete: 2802
Business Exception: 117
Locked: 10
Pending 54
Exception 814
